In [1]:
import os
os.getcwd()

'C:\\Users\\Admin\\data_cleaning_agent'

In [2]:
os.listdir()

['.ipynb_checkpoints',
 '01_data_profiling.ipynb',
 'adult.data',
 'adult.names',
 'adult.test',
 'Index',
 'old.adult.names']

In [3]:
import pandas as pd

In [4]:
columns = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week",
    "native_country", "income"
]

df = pd.read_csv(
    "adult.data",
    names=columns,
    na_values="?",
    skipinitialspace=True
)

df.head()


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
df.shape

(32561, 15)

In [6]:
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
income               0
dtype: int64

In [7]:
total_rows = len(df)
empty_rows = df.isna().all(axis=1).sum()
duplicate_rows = df.duplicated().sum()
total_rows, empty_rows, duplicate_rows

(32561, 0, 24)

In [8]:
df['age'].head(10)

0    39
1    50
2    38
3    53
4    28
5    37
6    49
7    52
8    31
9    42
Name: age, dtype: int64

In [9]:
df['age'].describe()

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [10]:
age_missing_pct = df['age'].isnull().mean()
age_unique_ratio = df['age'].nunique() / len(df)
age_missing_pct, age_unique_ratio 

(0.0, 0.0022419458861828567)

In [11]:
from dateutil.parser import parser

In [12]:
def detect_type(series):
    non_null = series.dropna()

    if len (non_null) == 0:
        return "unknown", 0.0, False
        
    numeric_ratio = pd.to_numeric(non_null, errors="coerce").notna().mean()

    unique_ratio = non_null.nunique() / len(series)
    suspected_id = unique_ratio > 0.95 and numeric_ratio > 0.95

    if numeric_ratio > 0.9:
        return "numeric", round(numeric_ratio, 2), suspected_id
    else: 
        return "text", 0.6, suspected_id
    

In [13]:
detect_type(df["age"])

('numeric', 1.0, False)

In [14]:
# More than 90% of values can be parsed as numbers, so the column is confidently numeric.

In [15]:
(df["age"]< 0).any(), (df["age"] > 120).any()

(False, False)

In [16]:
age_profile = {
    "column": "age",
    "type": "numeric",
    "missing_pct": round(age_missing_pct, 3),
    "unique_ratio": round(age_unique_ratio, 3),
    "suspected_id": False,
    "flags": []
}

age_profile

{'column': 'age',
 'type': 'numeric',
 'missing_pct': 0.0,
 'unique_ratio': 0.002,
 'suspected_id': False,
 'flags': []}

In [17]:
df["workclass"].head(15)

0            State-gov
1     Self-emp-not-inc
2              Private
3              Private
4              Private
5              Private
6              Private
7     Self-emp-not-inc
8              Private
9              Private
10             Private
11           State-gov
12             Private
13             Private
14             Private
Name: workclass, dtype: object

In [18]:
workclass_missing_pct = df["workclass"].isna().sum()
workclass_missing_pct

1836

In [19]:
df["workclass"].nunique(dropna=True)

8

In [20]:
workclass_unique_ratio = df["workclass"].nunique(dropna=True) / len(df)
workclass_unique_ratio

0.0002456926998556555

In [21]:
detect_type(df["workclass"])

('text', 0.6, False)

In [23]:
df["workclass"].value_counts(dropna=False)

workclass
Private             22696
Self-emp-not-inc     2541
Local-gov            2093
NaN                  1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

In [25]:
workclass_profile = {
    "column": "workclass",
    "type": "categorical",
    "missing_pct": round(workclass_missing_pct, 3),
    "unique_ratio": round(workclass_unique_ratio, 3),
    "suspected_id": False,
    "flags": ["has_missing_values"]
}

workclass_profile

{'column': 'workclass',
 'type': 'categorical',
 'missing_pct': 1836,
 'unique_ratio': 0.0,
 'suspected_id': False,
 'flags': ['has_missing_values']}

In [26]:
df["fnlwgt"].head(10)

0     77516
1     83311
2    215646
3    234721
4    338409
5    284582
6    160187
7    209642
8     45781
9    159449
Name: fnlwgt, dtype: int64

In [27]:
df["fnlwgt"].describe()

count    3.256100e+04
mean     1.897784e+05
std      1.055500e+05
min      1.228500e+04
25%      1.178270e+05
50%      1.783560e+05
75%      2.370510e+05
max      1.484705e+06
Name: fnlwgt, dtype: float64

In [28]:
fnlwgt_missing_pct = df["fnlwgt"].isnull().mean()
fnlwgt_unique_ratio = df["fnlwgt"].nunique() / len(df)

fnlwgt_missing_pct, fnlwgt_unique_ratio

(0.0, 0.6648444458094039)

In [30]:
detect_type(df["fnlwgt"])

('numeric', 1.0, False)

In [31]:
(df["fnlwgt"] < 0).any()

False

In [32]:
fnlwgt_profile = {
    "column": "fnlwgt",
    "type": "numeric",
    "missing_pct": round(fnlwgt_missing_pct, 3),
    "unique_ratio": round(fnlwgt_unique_ratio, 3),
    "suspected_id": True,
    "flags": ["high_uniqueness", "do_not_impute", "exclude_from_outliers"]
}

fnlwgt_profile

{'column': 'fnlwgt',
 'type': 'numeric',
 'missing_pct': 0.0,
 'unique_ratio': 0.665,
 'suspected_id': True,
 'flags': ['high_uniqueness', 'do_not_impute', 'exclude_from_outliers']}

In [49]:
df["education"].head(10)

0    Bachelors
1    Bachelors
2      HS-grad
3         11th
4    Bachelors
5      Masters
6          9th
7      HS-grad
8      Masters
9    Bachelors
Name: education, dtype: object

In [34]:
education_missing_pct = df["education"].isnull().mean()
education_missing_pct

0.0

In [36]:
df["education"].nunique()

16

In [37]:
education_unique_ratio = df["education"].nunique() / len(df)
education_unique_ratio

0.000491385399711311

In [39]:
df["education"].value_counts()

education
HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: count, dtype: int64

In [41]:
detect_type(df["education"])

('text', 0.6, False)

In [42]:
education_profile = {
    "column": "education",
    "type": "categorical",
    "missing_pct": round(education_missing_pct, 3),
    "unique_ratio": round(education_unique_ratio, 3),
    "suspected_id": False,
    "flags": ["paired_with_education_num"]
}

education_profile

{'column': 'education',
 'type': 'categorical',
 'missing_pct': 0.0,
 'unique_ratio': 0.0,
 'suspected_id': False,
 'flags': ['paired_with_education_num']}

In [48]:
df["education_num"].head(10)

0    13
1    13
2     9
3     7
4    13
5    14
6     5
7     9
8    14
9    13
Name: education_num, dtype: int64

In [55]:
edu_num_missing_pct = df["education_num"].isnull().mean()
edu_num_unique_ratio = df["education_num"].nunique() / len(df)

edu_num_missing_pct, edu_num_unique_ratio

(0.0, 0.000491385399711311)

In [51]:
df["education"].value_counts()

education
HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: count, dtype: int64

In [53]:
detect_type(df["education_num"])

('numeric', 1.0, False)

In [56]:
(df["education_num"] < 1).any(), (df["education_num"] > 20).any()

(False, False)

In [57]:
pd.crosstab(df["education"], df["education_num"])

education_num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
education,,,,,,,,,,,,,,,,
10th,0,0,0,0,0,933,0,0,0,0,0,0,0,0,0,0
11th,0,0,0,0,0,0,1175,0,0,0,0,0,0,0,0,0
12th,0,0,0,0,0,0,0,433,0,0,0,0,0,0,0,0
1st-4th,0,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5th-6th,0,0,333,0,0,0,0,0,0,0,0,0,0,0,0,0
7th-8th,0,0,0,646,0,0,0,0,0,0,0,0,0,0,0,0
9th,0,0,0,0,514,0,0,0,0,0,0,0,0,0,0,0
Assoc-acdm,0,0,0,0,0,0,0,0,0,0,0,1067,0,0,0,0
Assoc-voc,0,0,0,0,0,0,0,0,0,0,1382,0,0,0,0,0


In [59]:
education_num_profile = {
    "column": "education_num",
    "type": "numeric",
    "missing_pct": round(edu_num_missing_pct, 3),
    "unique_ratio": round(edu_num_unique_ratio, 3),
    "suspected_id": False,
    "flags": ["paired_with_education", "ordinal"]
}

education_num_profile

{'column': 'education_num',
 'type': 'numeric',
 'missing_pct': 0.0,
 'unique_ratio': 0.0,
 'suspected_id': False,
 'flags': ['paired_with_education', 'ordinal']}

In [61]:
df["occupation"].head(20)

0          Adm-clerical
1       Exec-managerial
2     Handlers-cleaners
3     Handlers-cleaners
4        Prof-specialty
5       Exec-managerial
6         Other-service
7       Exec-managerial
8        Prof-specialty
9       Exec-managerial
10      Exec-managerial
11       Prof-specialty
12         Adm-clerical
13                Sales
14         Craft-repair
15     Transport-moving
16      Farming-fishing
17    Machine-op-inspct
18                Sales
19      Exec-managerial
Name: occupation, dtype: object

In [62]:
occupation_missing_pct = df["occupation"].isnull().mean()
occupation_missing_pct

0.056601455729246644

In [64]:
df["occupation"].nunique(dropna=True)

14

In [65]:
occupation_unique_ratio = df["occupation"].nunique(dropna=True) / len(df)
occupation_unique_ratio

0.00042996222474739717

In [66]:
df["occupation"].value_counts(dropna=False)

occupation
Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales                3650
Other-service        3295
Machine-op-inspct    2002
NaN                  1843
Transport-moving     1597
Handlers-cleaners    1370
Farming-fishing       994
Tech-support          928
Protective-serv       649
Priv-house-serv       149
Armed-Forces            9
Name: count, dtype: int64

In [67]:
detect_type(df["occupation"])

('text', 0.6, False)

In [68]:
occupation_profile = {
    "column": "occupation",
    "type": "categorical",
    "missing_pct": round(occupation_missing_pct, 3),
    "unique_ratio": round(occupation_unique_ratio, 3),
    "suspected_id": False,
    "flags": ["has_missing_values", "high_cardinality"]
}

occupation_profile

{'column': 'occupation',
 'type': 'categorical',
 'missing_pct': 0.057,
 'unique_ratio': 0.0,
 'suspected_id': False,
 'flags': ['has_missing_values', 'high_cardinality']}

In [69]:
df["capital_gain"].head(20)

0      2174
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8     14084
9      5178
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
Name: capital_gain, dtype: int64

In [70]:
df["capital_gain"].head(20)

0      2174
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8     14084
9      5178
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
Name: capital_gain, dtype: int64

In [71]:
cap_gain_missing_pct = df["capital_gain"].isnull().mean()
cap_gain_unique_ratio = df["capital_gain"].nunique() / len(df)

cap_gain_missing_pct, cap_gain_unique_ratio

(0.0, 0.003654678910352876)

In [72]:
(df["capital_gain"] == 0).mean()

0.9167101747489328

In [73]:
detect_type(df["capital_gain"])

('numeric', 1.0, False)

In [74]:
df["capital_gain"].value_counts().head(10)

capital_gain
0        29849
15024      347
7688       284
7298       246
99999      159
5178        97
3103        97
4386        70
5013        69
8614        55
Name: count, dtype: int64

In [75]:
df["capital_gain"].sort_values(ascending=False).head(10)

17644    99999
30913    99999
20055    99999
6035     99999
1826     99999
24200    99999
24105    99999
6002     99999
24067    99999
11485    99999
Name: capital_gain, dtype: int64

In [76]:
capital_gain_profile = {
    "column": "capital_gain",
    "type": "numeric",
    "missing_pct": round(cap_gain_missing_pct, 3),
    "unique_ratio": round(cap_gain_unique_ratio, 3),
    "suspected_id": False,
    "flags": [
        "zero_inflated",
        "extreme_right_skew",
        "outliers_present",
        "do_not_impute_zeros"
    ]
}

capital_gain_profile

{'column': 'capital_gain',
 'type': 'numeric',
 'missing_pct': 0.0,
 'unique_ratio': 0.004,
 'suspected_id': False,
 'flags': ['zero_inflated',
  'extreme_right_skew',
  'outliers_present',
  'do_not_impute_zeros']}

In [77]:
df["capital_loss"].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: capital_loss, dtype: int64

In [78]:
df["capital_loss"].describe()

count    32561.000000
mean        87.303830
std        402.960219
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       4356.000000
Name: capital_loss, dtype: float64

In [79]:
cap_loss_missing_pct = df["capital_loss"].isnull().mean()
cap_loss_unique_ratio = df["capital_loss"].nunique() / len(df)

cap_loss_missing_pct, cap_loss_unique_ratio

(0.0, 0.0028254660483400386)

In [80]:
(df["capital_loss"] == 0).mean()

0.9533490986149074

In [81]:
detect_type(df["capital_loss"])

('numeric', 1.0, False)

In [82]:
df["capital_loss"].value_counts().head(10)

capital_loss
0       31042
1902      202
1977      168
1887      159
1848       51
1485       51
2415       49
1602       47
1740       42
1590       40
Name: count, dtype: int64

In [83]:
df["capital_loss"].sort_values(ascending=False).head(10)

14756    4356
8963     4356
30781    4356
23802    3900
20416    3900
11902    3770
15942    3770
12600    3683
23054    3683
30539    3004
Name: capital_loss, dtype: int64

In [84]:
capital_loss_profile = {
    "column": "capital_loss",
    "type": "numeric",
    "missing_pct": round(cap_loss_missing_pct, 3),
    "unique_ratio": round(cap_loss_unique_ratio, 3),
    "suspected_id": False,
    "flags": [
        "zero_inflated",
        "right_skewed",
        "outliers_present",
        "do_not_impute_zeros"
    ]
}

capital_loss_profile

{'column': 'capital_loss',
 'type': 'numeric',
 'missing_pct': 0.0,
 'unique_ratio': 0.003,
 'suspected_id': False,
 'flags': ['zero_inflated',
  'right_skewed',
  'outliers_present',
  'do_not_impute_zeros']}

In [85]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [86]:
df["native_country"].head(20)

0     United-States
1     United-States
2     United-States
3     United-States
4              Cuba
5     United-States
6           Jamaica
7     United-States
8     United-States
9     United-States
10    United-States
11            India
12    United-States
13    United-States
14              NaN
15           Mexico
16    United-States
17    United-States
18    United-States
19    United-States
Name: native_country, dtype: object

In [87]:
df["native_country"].head(20)

0     United-States
1     United-States
2     United-States
3     United-States
4              Cuba
5     United-States
6           Jamaica
7     United-States
8     United-States
9     United-States
10    United-States
11            India
12    United-States
13    United-States
14              NaN
15           Mexico
16    United-States
17    United-States
18    United-States
19    United-States
Name: native_country, dtype: object

In [88]:
country_missing_pct = df["native_country"].isnull().mean()
country_missing_pct

0.0179048555019809

In [89]:
df["native_country"].nunique(dropna=True)

41

In [90]:
country_unique_ratio = df["native_country"].nunique(dropna=True) / len(df)
country_unique_ratio

0.0012591750867602346

In [91]:
df["native_country"].value_counts(dropna=False).head(15)

native_country
United-States    29170
Mexico             643
NaN                583
Philippines        198
Germany            137
Canada             121
Puerto-Rico        114
El-Salvador        106
India              100
Cuba                95
England             90
Jamaica             81
South               80
China               75
Italy               73
Name: count, dtype: int64

In [92]:
detect_type(df["native_country"])

('text', 0.6, False)

In [93]:
native_country_profile = {
    "column": "native_country",
    "type": "categorical",
    "missing_pct": round(country_missing_pct, 3),
    "unique_ratio": round(country_unique_ratio, 3),
    "suspected_id": False,
    "flags": [
        "has_missing_values",
        "high_cardinality",
        "skewed_distribution",
        "rare_categories_present"
    ]
}

native_country_profile

{'column': 'native_country',
 'type': 'categorical',
 'missing_pct': 0.018,
 'unique_ratio': 0.001,
 'suspected_id': False,
 'flags': ['has_missing_values',
  'high_cardinality',
  'skewed_distribution',
  'rare_categories_present']}

In [94]:
df["income"].value_counts(normalize=True)

income
<=50K    0.75919
>50K     0.24081
Name: proportion, dtype: float64